In [2]:
!pip install keras_applications

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 3.4 MB/s eta 0:00:00


In [6]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D


In [10]:
dataset_path = "/content/drive/MyDrive/TY VIIT/SEM V/DL/Assignments/Assignment 2/dataset"


In [11]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)


In [12]:
train_gen = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode="binary",
    subset="training"
)


Found 7571 images belonging to 2 classes.


In [13]:
val_gen = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode="binary",
    subset="validation"
)


Found 1892 images belonging to 2 classes.


In [14]:
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224,224,3))
base_model.trainable = False

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.5),
    Dense(1, activation="sigmoid")
])

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [15]:
history = model.fit(train_gen, validation_data=val_gen, epochs=5)


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
237/237 ━━━━━━━━━━━━━━━━━━━━ 1719s 7s/step - accuracy: 0.9068 - loss: 0.2046 - val_accuracy: 0.9995 - val_loss: 0.0126
Epoch 2/5
237/237 ━━━━━━━━━━━━━━━━━━━━ 138s 582ms/step - accuracy: 0.9978 - loss: 0.0162 - val_accuracy: 1.0000 - val_loss: 0.0061
Epoch 3/5
237/237 ━━━━━━━━━━━━━━━━━━━━ 136s 576ms/step - accuracy: 0.9994 - loss: 0.0077 - val_accuracy: 1.0000 - val_loss: 0.0055
Epoch 4/5
237/237 ━━━━━━━━━━━━━━━━━━━━ 150s 633ms/step - accuracy: 0.9989 - loss: 0.0061 - val_accuracy: 0.9989 - val_loss: 0.0049
Epoch 5/5
237/237 ━━━━━━━━━━━━━━━━━━━━ 136s 575ms/step - accuracy: 0.9998 - loss: 0.0035 - val_accuracy: 1.0000 - val_loss: 0.0032


In [18]:
model.save('/content/drive/MyDrive/TY VIIT/SEM V/DL/Assignments/Assignment 2/human_vs_nonhuman.h5')
print("Model saved to google drive")

Model saved to google drive


In [20]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load model
model = load_model("/content/drive/MyDrive/TY VIIT/SEM V/DL/Assignments/Assignment 2/human_vs_nonhuman.h5")

# Load and preprocess image
img_path = "/content/drive/MyDrive/TY VIIT/SEM V/DL/Assignments/Assignment 2/dataset/humans/Jennie_Garth_0001.jpg"
img = cv2.imread(img_path)
img = cv2.resize(img, (224, 224))
img = img.astype("float32") / 255.0
img = np.expand_dims(img, axis=0)

# Predict
pred = model.predict(img)[0][0]
if pred > 0.5:
    print("👤 Person Detected")
else:
    print("🚫 Non-Person Detected")


1/1 ━━━━━━━━━━━━━━━━━━━━ 12s 12s/step
🚫 Non-Person Detected
